# Hashtabellen

Die wichtigste Komponente, um einen Hashtabelle bauen zu können, ist die Hashfunktion, welche uns für jedes Objekt einen ganzzahligen Wert (den Hashwert) liefert. In Python berechnen wir dies mit der Funktion ```hash```. Für alle Standardtypen in Python ist die Hashfunktion bereits definiert.

In [ ]:
print("Hashwert für String abc", hash("abc"))
print("Hashwert für Integer 7", hash(7))
print("Hashwert für Tuple (\"abc\", 7)", hash(("abc",7)))

Wie wir Hashfunktionen für eigene Datentypen bauen können, und was wir dabei beachten müssen, wird in einem separaten Notebook behandelt. Für die nachfolgenden Implementationen gehen wir davon aus, dass die Hashfunktion für den Schlüssel bereits definiert ist. 

## Implementation von Hashtabellen

### Verkettung

Die erste Implementation von Hashtabellen die wir sehen, hebt Konflikte auf, indem für jedes Element im Array eine Verkettete Liste mit allen Schlüssel/Werte-paaren gespeichert wird. 
*Achtung: Wir nehmen hier eine fixe Grösse der Tabelle an. Sie können die resize Methoden als Übung hinzufügen.*

In [ ]:
import sys
class SeparateChainingHashtable:
    def __init__(self):
        self._M = 31 # Number of chains
        self._st = [None]*self._M # chains
        self._N = 0
    
    class Node:
        def __init__(self, key, value, next = None):
            self.key = key
            self.value = value
            self.next = next
      

    def _hash(self, key):
        positiveHash = hash(key) % ((sys.maxsize + 1) * 2) 
        return positiveHash % self._M
        
    def get(self, key):
        i = self._hash(key);
        x = self._st[i]
        while x != None:
            if key == x.key:
                return x.value
            x = x.next
        return None
    
    def put(self, key, value):
        if  self._N / self._M > 4:
            self.resize(self._M * 2)
        i = self._hash(key);
        x = self._st[i]
        while x != None:
            if key == x.key:
                x.value = value
                return
            x = x.next
        self._st[i] = SeparateChainingHashtable.Node(key, value, next = self._st[i]);
        self._N += 1
    
    def delete(self, key):
        if  self._N / self._M < 1/4:
            self.resize(self._M / 2)
        i = self._hash(key);
        
        x  = self._st[i]
        if x == None:
            return
        
        if key == x.key:
            self._st[i] = x.next   
            self._N -= 1
            return 
        
        while x.next != None:
            if key == x.next.key:
                x.next = x.next.next
                self._N -= 1
                return
        
    def contains(self, key):
        return get(key) != None
    
    def size(self):
        return self._N
    
    def isEmpty(self):
        return self.size() != None
    
    def keys(self):
        for st in self._st:
            x = st
            while x != None:
                yield x.key
                x = x.next
    
    # helper function to diagnose implementation
    def lengthOfInternalLists(self):
        lengths = []
        for st in self._st:
            lengths.append(self._length(st))
        return lengths
    
    def _length(self, st):
        n = 0
        x = st
        while x != None:
            n += 1
            x = x.next
        return n
    
    def resize(self, size):
        # copy key value pairs into a new list
        old = []
        for key in self.keys():
            old.append((key, self.get(key)))
        # resize and empty current table
        self._M = size
        self._st = [None] * self._M
        self._N = 0
        # add old key value pairs into new table
        for key, value in old:
            self.put(key, value)


In [ ]:
ht = SeparateChainingHashtable()
for (pos, c) in enumerate("SEARCHEXAMPLE"):
    ht.put(c, pos)

In [ ]:
for key in ht.keys():
    print(key, ht.get(key))

Wir sehen, dass im Gegensatz zu Binären Suchbäumen, die Schüssel hier nicht geordnet sind. 

Als nächstes fügen wir zufällige Elemente ein und schauen uns die Länge der internen Listen an.

In [ ]:
import random

ht = SeparateChainingHashtable()
for i in range(0, 100):
    ht.put(random.randint(0, 100000), "")
print(ht.lengthOfInternalLists())
print('longest list:', max(ht.lengthOfInternalLists()))
print("number of internal lists:", ht._M)

#### Übung: 
* Implementieren Sie die Methode ```resize```

## Lineares sondieren

In der zweiten Implementation verwenden wir *lineares Sondieren*. 

*Achtung, auch hier belassen wir die Grösse der Tabelle fix. Für eine praktikable Implementation müssten wir natürlich die Tabelle dynamisch vergrössern.*

In [ ]:
import copy

class LinearProbingHashtable:
    
    def __init__(self, printKeysOnInsert=False):
        self._M = 17 #7919
        self._keys = [None] * self._M
        self._values = [None] * self._M
        self._N = 0
        self._printKeys = printKeysOnInsert
   
    
    def _hash(self, key):
        positiveHash = hash(key) % ((sys.maxsize + 1) * 2) 
        return positiveHash % self._M

    def get(self, key):
        i = self._hash(key);
        
        while self._keys[i] != None:
            if (self._keys[i] == key):
                return self._values[i]
            i = (i + 1) % len(self._keys)        
    
    def put(self, key, value):
        if self._printKeys:
            print(self._keys)
            
        i = self._hash(key)
        while self._keys[i] != None:
            if self._keys[i] == key:
                break;
            i = (i + 1) % len(self._keys)  
        if self._keys[i] == None:
            self._N += 1;

        self._keys[i] = key;
        self._values[i] = value
            
    def size(self):
        return self._M

    def isEmpty(self):
        return self.size() != 0
    
    def contains(self, key):
        return self.get(key) != None
    
    def keys(self):
        for key in self._keys:
            if (key != None):
                yield key
            

Wenn wir in diese Tabelle einfügen und uns in jedem Schritt die Schlüssel ausgeben lassen, dann sehen wir das typische Clustering. 

In [ ]:
ht = LinearProbingHashtable(printKeysOnInsert=True)
for (pos, c) in enumerate("SEARCHEXAMPLE"):
    ht.put(c, pos)


#### Übung:
* Implementieren Sie die ```delete``` Methode